# PIA - Reinforcement Learning

1. RL Intro
2. **Dynamic Programing**
3. Tabular Q-Learning
4. Deep Reinforcement Learning (DQN)

## 2. Dynamic Programming

In this notebook you will implement a first solution method for the gridworld maze with Dynamic Programming. More precisely you will implement the Value Iteration algorithm and learn more about its central concept, the value function. Understanding the benefits and shortcommings of this approach is key to better understand all other RL algorithms, their solutions and trade-offs. 

---
### Course of Action

* Please write all executable python code in ```Code```-Cells (```Cell```->```Cell Type```->```Code```) and all Text as [Markdown](http://commonmark.org/help/) in ```Markdown```-Cells
* Describe your thinking and your decisions (where appropriate) in an extra Markdown Cell or via Python comments
* In general: discuss all your results and comment on them (are they good/bad/unexpected, could they be improved, how?, etc.). Furthermore, visualise your data (input and output).
* Write a short general conclusion at the end of the notebook
* Further experiments are encouraged. However, don't forget to comment on your reasoning.
* Use a scientific approach for all experiments (i.e. develop a hypothesis or concrete question, make observations, evaluate results)

### Submission

E-Mail your complete Notebook to [maucher@hdm-stuttgart.de](mailto:maucher@hdm-stuttgart.de) until the start of the next lecture. One Notebook per Group is enough. Edit the teammember table below.

**Important**: Also attach a HTML version of your notebook (```File```->```Download as```->```HTML```) in addition to the ```.ipynb```-File.

| Teammember |                    |
|------------|--------------------|
| 1.         | Geoffrey Hinton    |
| 2.         | Yoshua Bengio      |
| 3.         | Yann LeCun         |
| 4.         | Jürgen Schmidhuber |
---

## Prerequisites

- The notebook is pretty self contained and includes all relevant theory. However, make sure that you have **read** and *understood* a corresponding lecture at least once! The notebook can only provide a short recap. Being prepared will help you to finish the notebook in time!
- Some good reading resources are:

    1. Johannes Maucher's [lecture on RL](https://gitlab.mi.hdm-stuttgart.de/maucher/ML/tree/master/Slides/V12Reinforcement.pdf) - chapter 2 & 3.
    2. David Silver's course on RL: [Lecture 2](https://www.davidsilver.uk/teaching/): [Youtube](https://www.youtube.com/watch?v=lfHX2hHRMVQ), [Lecture 3](https://www.davidsilver.uk/teaching/): [Youtube](https://www.youtube.com/watch?v=Nd1-UUMVfz4).
    3. Sutton & Barto: [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) chapter 4 (complete online pdf draft available).

### Welcome back!

By now, you should be familiar with our little gridworld game and how to use it in an **agent-environment loop**. As a side effect you should have gathered vital knowledge about the environment dynamics e.g. the effect of actions and how rewards are emitted. Next, you will use this knowledge to implement a first solution method with **Dynamic Programming (DP)**.

> Why can't we just skip this and proceed to DQN?
>
> #### From *Reinforcement Learning: An Introduction* - Chapter 4: Dynamic Programming
>
> "DP provides an essential foundation for the understanding of the methods presented in the rest of this book. In fact, all of these methods can be viewed as attempts to achieve much the same effect as DP, only with less computation and without assuming a perfect model of the environment."

### Markov Decision Process

Great, now that we have "convinced" you that DP is worth studying :D , let us recap everything so far but this time a bit more formally. As mentioned earlier, our gridworld problem can be described as a **Markov Decision Process (MDP)**. Formally, a MDP is a 5-tuple $\big \langle \mathcal{S,A,P,R,\gamma}  \big \rangle$, where

* $\mathcal{S}$ - is a finite set of **states**.
* $\mathcal{A}$ - is a finite set of **actions**.
* $\mathcal{P}$ - are the **state transition probabilities**. These define the dynamics of our environment which can be stochastic or deterministic.
* $\mathcal{R}$ - is the **reward function**. It determines how much reward is given for a specific (action, state transition) pair. Again, this can be stochastic or deterministic.
* $\mathcal{\gamma}$ - is a discount factor $0 \leq \gamma \leq 1$ that helps us to model the importance of short term vs. long term reward. Most commonly $\gamma = 0.9$.

Furthermore, **time** is modeled as discrete timesteps $t = 1,2,3,\ldots$. If there exists a timestep $T < \infty$ in which the environment reaches a **terminal state**  $S_T$, then the system is called **episodic MDP**. This stands in contrast to the **continuous** case where no such naturally occuring state exists. A similar distinction can be made for the state and action space. When you read something like *"Continuous control for Deep Reinforcement Learning"*, the authors typically refer to a continuous state or action space and not the time.

### Gridworld MDP
Let us make this more concrete by applying the MDP framework to our gridworld problem.

In [ ]:
from gridworlds import exitgames
game, ui    = exitgames.make_game(level=0), exitgames.get_notebook_ui()
obs, rew, _ = game.its_showtime()

game.print_action_meanings()
ui.render(obs, "COLOR")

In the case of `exitgames.make_game(level=0)`:

* we have $4 \times 4$ discrete states $s \in \mathcal{S}$.
* we have $5$ discrete actions  $a \in \mathcal{A} = \big\{NORTH,SOUTH,WEST,EAST,STAY\big\}$.
* $\mathcal{P}(s,a)$ will produce a next state $s'$ whereby every action will succeed with probability $\mathbb{P} = 1$.  
* $\mathcal{R}(s,a,s')$ will produce a reward $r \in \{-1,0,1\}$ with probability $\mathbb{P} = 1$. 

Because every action will result in the desired next state and every reward will be emitted with probability 1, our gridworld is a **deterministic MDP**. Because the sets of actions, states and possible rewards are finite, the systems is a **finite MDP**. Finally, the *exit* square depicts a terminal state from where no further improvement is possible, hence our exitgame is an **episodic MDP**.

> It is always good to formalize your problem in this way! Even though your knowledge about the environment is typically much more limited (or the environment is much more complex), it helps you to understand its basic properties and gives you a first estimate of its complexity. In consequence, it helps you to make informed decisions about the family of algorithms you want/have to apply.

### Policies and Value Functions

Now before we start implementing our first solution, let us quickly recap two more concepts that are central to all algorithms we discuss in this lab. 

##### The Policy
The first one is the so called **policy** $\pi$. It is basically a mapping from states to actions and **fully describes the behavior of an agent**. In the most simple, deterministic case this can be written as $\pi_t(s) = a$. Think of a list of rules that literally is holding a state -> action pair for every state. More common in reinforcement learning is the usage of a **stochastic policy** which defines **the probability of choosing a distinct action $a$ given a state $s$**. Think of the random agent you just implemented. In every state the agent has chosen an action according to a **discrete uniform distribution** whereby every action is equally likely to be picked. Formally such policies are defined as

$$\pi_t(a \mid s) = \mathbb{P}\big[A_t = a \mid S_t = s\big]$$

For the discrete case, this can again be implemented as a simple list but now, holding the probabilites for every action in every state. 


#####  Value Functions
The second concept is the so called **value function** $V$. The idea is simple, $V(s)$ tells us *how good* it is to be in a given state $s$. More precisely, it tells us **how much future reward we can expect** when starting from that specific state. The amount of future reward obviously depends on the policy which the agent is following, so formally we write $V_{\pi}(s)$. In summary,

> $V_{\pi}(s)$ tells us how much reward we can expect when starting in state $s$ and following policy $\pi$ thereafter.

Because $V$ assigns values only to states, it is called the **state-value function**. Later we will also use the so called **action-value function** $Q(s,a)$ which estimates the future reward for every state-action pair separately. In other words,

> $Q_{\pi}(s,a)$ tells us how much reward we can expect when starting in state $s$, taking action $a$ and following policy $\pi$ thereafter.

### Gridworld Policy and Value Function

Again, let us make this more concrete for our little gridworld.

First, create a state-value function `V` in the form of an $n \times n$ dimensional numpy array and **initialize it with zeros**. We will use this kind of table to store and update all state values. The states itself are now implicitly described by the array indices. So the "first" state would be something like `row, col = 0, 0` and the value for that state can be retrieved by *calling* the value function `V[row,col]`, i.e. make an array lookup at this index. Now create and plot this thing with the code below.

In [ ]:
import numpy as np
import seaborn as sns

def plot_value_function(V, print_values=True):
    ax = sns.heatmap(V, vmin=-1.5, vmax=1.5, annot=print_values, 
                        xticklabels=True, yticklabels=True, 
                        square=True, linewidths=.5,  
                        cmap=sns.diverging_palette(10, 220, sep=40, n=256))
    return ax

In [ ]:
V = # Your code comes here.   
plot_value_function(V)

Second, create a policy `Pi` in the form of an $n \times n \times a$ dimensional numpy array. Here $a$ should be the number of actions. For simplicity, initialize the array with zeros again. In this case, the indices now refer to state followed by the action and the numbering of actions corresponds to the output of `game.get_action_set()`. So, the probability for taking action `0` in the "first" state can be retrieved by calling `Pi[0,0,0]`.

Now think about our current value function. With the knowledge that we have so far, what kind of policy would you reccomend based on the current state values? You can use `Pi.fill()` to set a value for every entry in the array at once.

In [ ]:
Pi = # Your code starts here.

Plotting the policy is not as straight forward as it was for the value function. However, here is a function that you can use to plot the policy as little arrows on top of the value function. Remember that our actions are essentially vectors like `(-1, 0), (1, 0)` etc.? The code bellow will just use this representation to plot corresponding arrow directions. Now go ahead and visualize your policy.

In [ ]:
def plot_policy(V, Pi, action_set):
    # plot the value funtion as a background
    ax = plot_value_function(V, print_values=False)
    
    # plot every action in a quiver arrow plot
    num_rows, num_cols, num_actions = Pi.shape   
    for row in range(num_rows):
        for col in range(num_cols):
            for action in range(num_actions):
                probability = Pi[row,col,action]

                if probability: # if not zero
                    # Note that action directions and array indices are [row,col] based which corresponds to [y,x] in 
                    # cartesian coordinates. Quiver plot works with cartesian coordiantes!
                    # Note further that the sns heatmap plot has inverted the y axis which is fine as long as we are
                    # adding our arrows to the same axis.
                    move_y, move_x = action_set[action]
                    length = 2 + 2 * (1 - probability)
                    ax.quiver(col+0.5, row+0.5, move_x, move_y, angles='xy', scale_units='xy', scale=length)

In [ ]:
action_set = game.get_action_set()
plot_policy(V, Pi, action_set)

Ok great, we now have an explicit version of our random agent! The good news is that we can now update specific states and actions as we like. Next, let us make use of this new *feature* in order to improve our policy and make it more sane. The first thing we definetly want to improve is, to consider only reasonable moves, i.e. we want to ignore moves that will cause the agent walking into a wall etc.

### Building a world model

In the following you will build a world model using a map of the environment and the knowledge that you have. More precisely:

- consider only reasonable actions for the policy.
- implement $\mathcal{P}(s,a)$
- implement $\mathcal{R}(s,a,s')$

Here is the idea. The function `game.get_world_map()` will return you a plain version of the environment map without the player character. Since this map is a 2-dimensional array, you can query specific states of the map in the same way as we represent our states in the value function, i.e. with indices. For instance you can use this map to check if a specific state contains an "x" or "E" character etc. However, note that our value function is of shape $4 \times 4$ whereas the world map also contains the walls and is of shape $6 \times 6$. If you now loop over all value function indices (states) make sure your add (1,1) to it in order to translate to the corresponding world position. 

Next, check out the world map and try to query the "exit" square which is position `(0,3)` in state space. If you want, use the `ui.render` function to visualize the map.

In [ ]:
world_map = game.get_world_map()
# Your code comes here...

We can now use this idea to limit the action set for a specific state to more reasonable actions, e.g. ignore those actions that will just walk us into a wall. To do so we can take a specific world position, add the value of an action to this tuple and check if the world map contains a wall character at the new position. The function `game.allowed_actions` does exactly this. It takes a *world position* in the form of a *tuple* and will return a reduced action set for that state.

Now use this function to create a more sane initial random policy. Distribute the probabilities equally across all allowed actions and set all other actions to zero. Remember that the action keys are `0,1,2,3,4` which corresponds to the their index in the policy array. Here is some initial code you can complete. Print and plot the policy afterwards.

In [ ]:
def random_policy():

    Pi = np.zeros((4,4,5))
    num_rows, num_cols, num_actions = Pi.shape   
    for row in range(num_rows):
        for col in range(num_cols):
            
            # Your code comes here...
            # 1. Get the allowed actions for a distinct state
            # 2. Set a fitting probability for the allowed actions
    
    return Pi

In [ ]:
Pi = random_policy()
#print(Pi)
plot_policy(V, Pi, action_set)

Next, let us implement the **state transition function** $\mathcal{P}(s,a)$. For clarity, we will just call it `get_next_state`. Since our environment is deterministic, going from one state to another can be calculated by simply adding an action to a state tuple and return the resulting next state indices. 

Note that this state transition fully operates in value function space so performing an action `(-1,0)` in state `(0,0)` would result in a next state `(-1,0)` which is not *allowed*! However, since we will consider only allowed actions in our policy, you don't have to check for this case again. Just keep it very simple here.

But Beware! There is **one** important exception to this. Note that the exit square is a terminal state and that we have to reset the game from there. To model this correctly, make sure to return the starting position as the "next state" for the terminal state.

In [ ]:
def get_next_state(state, action):
    
    # Your code comes here...
    next_state = 
    
    return next_state

In [ ]:
# Testing cell

Finally we have to implement the **reward function** $R(s,a,s')$. Again, we will just call it `get_reward`. Since our environment is also deterministic for rewards, this function boils down to a simple lookup on the world map and should return a corresponding reward value. In general the reward may depend on the full triple $(s,a,s')$ but in our case it doesn't matter from which state we are coming or which action we have performed last. The only thing we care is the reward value for *being* in a distinct state $s$.

> Hint: you can use `ord()` to translate unicode characters to their integer code point value or `chr()` for the inverse.

In [ ]:
def get_reward(state, world_map):
    # Your code comes here...
    reward = 

    return reward

In [ ]:
# Testing cell

### Policy Evaluation

Great, now we have everything in place! In the remainder of this notebook you will finally implement an actual solution method called **Policy Iteration**. Policy Iteration consist of two parts. First, we will use **policy evaluation** to estimate the value of a given policy. Second we will use **policy improvement** to update our policy towards a better policy. Evaluating and improving a policy over and over will finally converge to a so called **optimal policy** $\pi_*$ which is defined as a policy that is better or equal to all other policies. In consequence, all optimal policies will produce (or share) the same **optimal state-value function** $v_*(s) = \displaystyle \max_\pi v_{\pi}(s)$. In reverse, if we can find an optimal value function we can derive an optimal policy by simply acting greedy with respect to the value function, i.e. by picking the states and actions with the highest values.

Now recap that with *better*, we mean a policy that will produce more reward in the long-run which we called **return**. Formally, we consider the *discounted return*,

$$\begin{eqnarray} 
G_t &=& R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots = \displaystyle \sum_{k=0}^{\infty}\gamma^k R_{t+k+1} \\
    &=& R_{t+1} + \gamma G_{t+1}
\end{eqnarray} $$

where $\gamma$ is the *discount rate* $0 \leq \gamma \leq 1$ and determines the importance of short term vs. long term reward. We will set $\gamma = 0.9$ from this point on. Note that by recursion we can write the return as the immediate reward plus the discounted return at timestep $t+1$. Next recap that the value function $V$ estimates the expected return when starting in a state $s$ and following policy $\pi$ thereafter, formally

$$V_{\pi}(s) = \mathbb{E}_{\pi} \Big[G_t \mid S_t = s\Big], \text{for all } s \in \mathcal{S}$$

By the same recursive relationship we can unroll the equation as follows,

$$\begin{eqnarray} 
V_{\pi}(s) &=& \mathbb{E}_{\pi} \Big[G_t \mid S_t = s\Big] \\
           &=& \mathbb{E}_{\pi} \Big[R_{t+1} + \gamma G_{t+1} \mid S_t = s \Big] \\
           &=& \mathbb{E}_{\pi} \Big[R_{t+1} + \gamma V_{\pi}(S_{t+1}) \mid S_t = s \Big] \\
\end{eqnarray}$$

In words, the value function of state $s$ is the expected immediate reward plus the discounted value of the expected next state. The tricky part of the equation is the expectation $\mathbb{E}$ which accounts for the potential stochastic policy and environent dynamics. In order to calculate $V(s)$ we have to sum over all actions that may be taken under a certain policy (the first sum below) and then sum over the distribution of rewards and next state values that the environment may produce (the second sum below). Formally, 

$$\begin{eqnarray} 
V_{\pi}(s) &=& \displaystyle \sum_a \pi(a\mid s) \sum_{s',r} p(s',r\mid s,a) \Big[r + \gamma V_{\pi}(s')\Big]
\end{eqnarray}$$

where $\pi(a \mid s)$ is the probability of taking action $a$ in state $s$ under policy $\pi$. We have stored this probability in our policy array. Furthermore we are solving a deterministic environment so the combined probability distribution $p(s',r \mid s,a)$, or the second sum, simplifies to a single case that will occur with probability 1. Since we have a model of the world we can get the values for $r$ and $s'$ by calling `get_reward` and `get_next_state` and the value for $V_{\pi}(s')$ by a simple table lookup in the current value function $V$. In summary, we can evaluate a policy by iteratively calculating 

$$\begin{eqnarray} 
V_{\pi}(s) &=& \displaystyle \sum_a \pi(a\mid s) \Big[\mathcal{R}(s) + \gamma V_{\pi}(\mathcal{P}(s,a))\Big]
\end{eqnarray}$$

for all states. This approach is called **iterative policy evaluation**. Now let us translate this into code.

### Policy Evaluation implementation

Here is a short checklist :)

1. Create a new value function `V_new` (that is all zero)
2. For every `state`
    - initialize a sum placeholder `v = 0`
    - translate the state to a world position
    - get the set of possible actions by calling `game.allowed_actions`
    - for every possible action
        - get the **probability** for taking the action from `Pi`
        - get the **reward** for the current state by calling `get_reward`
        - get the **next state** by calling `get_next_state`
        - calculate and add to partial `v` according to the equation above
    - set `V_new[state] = v`
3. Return the new value function

In [ ]:
def evaluate_policy(V, Pi, gamma):

    # Your code comes here...
    V_new = 

    return V_new

- Below is a code snippet that will plot the value function and the policy next to each other. 
- Below that, is a code snipet that you can use to easily test your implementation (as long as you obeyed our naming conventions).

In [ ]:
import matplotlib.pyplot as plt

def plot_things(Vi, Pi, action_set):
    plt.close()
    fig = plt.figure(figsize=(13,4.5))
    
    # plot value function left
    fig.add_subplot(1,2,1)
    plot_value_function(V)

    # plot policy right
    fig.add_subplot(1,2,2)
    plot_policy(V,Pi,action_set)

In [ ]:
# initialize a clean value function , a random policy, gamma and get the action_set
V = np.zeros((4,4))
Pi = random_policy()
gamma = 0.9
action_set = game.get_action_set()

# evaluate the policy once and plot the result
V = evaluate_policy(V, Pi, gamma)
plot_things(V,Pi,action_set) 

If you feel comfortable that your implementation works here is another code snippet that lets you see the evaluation process over time. Try different variants for `n` to see the value function converge. Comment/Uncomment the display related code accordingly.

In [ ]:
from IPython import display

# initialize a clean value function and random policy
V, Pi, gamma, action_set = np.zeros((4,4)), random_policy(), 0.9, game.get_action_set()

# evaluate the policy n times
for n in range(5):
    V = evaluate_policy(V, Pi, gamma)
    plot_things(V,Pi,action_set)
    plt.pause(0.3)
    display.clear_output(wait=True)

### Policy Improvement

Now that we can evaluate our policy, let's improve it based on the estimate of the action values function. We can do this by acting greedy w.r.t to the action values. In words, in every state we want to select those actions (from all possible actions) that appear best according to the **action-value function** $Q_{\pi}(s,a)$. Formally we want to retrieve a new *greedy* policy $\pi'$, give by

$$\begin{eqnarray} 
\pi'(s) &=& \arg \max_a Q_{\pi} \big(s,a\big) \\
        &=& \arg \max_a \mathbb{E} \Big[R_{t+1} + \gamma V_{\pi}(S_{t+1}) \mid S_t = s, A_t = a \Big] \\
        &=& \arg \max_a \displaystyle \sum_{s',r} p(s',r \mid s,a) \Big[r + \gamma V_{\pi}(s')\Big]                
\end{eqnarray}$$

which in our case can be calculated by

$$
\pi'(s) = \arg \max_a \displaystyle \Big[\mathcal{R}(s) + \gamma V_{\pi}(\mathcal{P}(s,a))\Big]
$$

This looks very similar to the calculation of the state-value function. However, **we now compare the outcome of every action separately** and select only the best one, i.e. the action that appears to return the most reward in the long run. If there is a single best action, the new greedy policy will *choose* that action with probability 1 in the future. If there are more than one equally *good* actions the probability mass should be distributed equally between them.

To make the difference more clear. We now ignore the probability under which some action $a$ was taken given some old policy $\pi$! Instead, we want to create a **new policy** $\pi'$ and we do this by selecting the most promising action **from all possible actions**. 


### Policy Improvement implementation

Again, use the code cells below to test your implementation.
Here is a short checklist :)

1. Create a new policy `Pi_new` (that is all zeros)
2. For every `state`
    - initialize a placeholder for the best value so far `q_max`
    - initialize an empty list `best_actions = []`
    - translate the state to a world position
    - get the set of possible actions by calling `game.allowed_actions`
    - for every possible action
        - get the **reward** for the current state by calling `get_reward`
        - get the **next state** by calling `get_next_state`
        - calculate `q(s,a)` according to the equation above
        - check if `q` is greater or equal to `q_max` and store the action accordingly in `best_actions`.   
    
    - distribute the new probabilities equally over all `best_actions`
    - assign zero probability to all other actions
3. Return the new policy

In [ ]:
def greedy_policy(V, gamma):

    # Your code comes here...
    Pi_new = 
                    
    return Pi_new

In [ ]:
# initialize a clean value function and random policy
V, Pi, gamma, action_set = np.zeros((4,4)), random_policy(), 0.9, game.get_action_set()

# evaluate the policy once
V  = evaluate_policy(V, Pi, gamma)

# update the policy
Pi = greedy_policy(V, gamma)

# plot the value function and the updated policy
plot_things(V,Pi,action_set)

If your are convinced that your greedy policy is working, proceed to the next cell.

### Value Iteration

Cool. Now you can use iterative policy evaluation and greedy policy improvement in a loop to solve our little gridworld. In general, this algorithm is called **Policy Iteration**. Here are some completing questions:

- Do you think it is necessary for the value function to converge before you can apply a meaningful policy improvement?
- If not, how many evaluation steps do you think are enough for the algorithm to work?

Okay, these are intentionally very leading questions. There exists a popular special case of policy iteration in which the policy gets improved after **every** evaluation step. This variant is known as the **Value Iteration** algorithm. If you have sticked to our naming conventions, you can use the code below to run your implementation and watch your value function and policy converging over time. Some final remarks:

- Make sure you try different values for `k`.
- How many iterations does it take for the **policy** to converge?
- Why do you think does it take exactly that many steps?
- To print all iterations, outcomment the `display.clear_output` call.



- (Optionally) play with different values for `n` and `gamma`.

In [ ]:
# initialize a clean value function and a random policy
V, Pi      = np.zeros((4,4)), random_policy() 
action_set = game.get_action_set()
gamma      = 0.9 

# run the policy iteration algorithm for k steps
for k in range(15):
    
    # evaluate the policy n times, n=1 is called value iteration
    for n in range(1):
        V  = evaluate_policy(V, Pi, gamma)
    
    # greedily improve the policy
    Pi = greedy_policy(V, gamma)
    
    # plot the evaluated value function and the updated policy
    plot_things(V,Pi,action_set)
    plt.pause(0.5)
    display.clear_output(wait=True)

## Further Ideas

Mandatory:
- See task list from **Value Iteration** and write down your thoughts.

Optionally: (only if good in time, if not, better start with the next notebook)

- Experiment with different settings. 
- Experiment with exitgames level 1.

## Next: PIA - Reinforcement Learning: 3. Tabular Q-Learning